# Proyecto 1.  

---

Universidad:    UTEC  
Curso:          Inteligencia Artificial-CS2601  
Profesor:       Cristian López Del Alamo  
Tema:           Regresión y *machine learning* para predicción de áreas de incendio forestal.  
Integrantes:
- Sebastián Lizárraga
- Carlos Guerrero

---



1.  En esta práctica se pide realizar pruebas utilizando   diferentes funciones de pérdida. 

2.  Su equipo debe implementar el algoritmo de machine learning para  regresión lineal múltiple y realizar las correspondientes pruebas usando el siguiente  [Dataset](https://drive.google.com/file/d/18uGca04UQTt6xR1BTfecZzqhFtp3TqKa/view?usp=sharing).  

3. MSE Loss Function \\

  $MSE = \frac{1}{2m}\sum_{i=0}^m (y_i - h(x_i))^2$ 

4. Utilize todo los datos del dataset para entrenar  y grafique el plano que mejor separa a los datos. [Help](https://stackoverflow.com/questions/36060933/matplotlib-plot-a-plane-and-points-in-3d-simultaneously)

Importante: No se olvide de normalizar los datos entre cero y uno, por cada columna.
 
 


Crear el DataSet

Data set info: https://www.kaggle.com/datasets/elikplim/forest-fires-data-set






In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Normalización

In [13]:
def normalice(df_input):
    z = pd.DataFrame(df_input)
    # Normalizacion con media y desviacion estandar
    return z.apply(lambda x: (x-x.mean())/ x.std(), axis=0)
    # Normalizacion con minimos y maximos
    # return (z - z.min()) / ( z.max() - z.min())

In [14]:
dataset = pd.read_csv('forestfires.csv')
X = dataset[["RH","temp"]]
Y = dataset[["area"]]
# Normalizamos la data
X = normalice(X)
Y = normalice(Y)
# Luego de normalizar agregamos la columna del bias
X = np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)

print(X)
print(Y)

[[ 1.          0.41132597 -1.84085723]
 [ 1.         -0.69178627 -0.15312996]
 [ 1.         -0.69178627 -0.73866799]
 ...
 [ 1.          1.57572222  0.39796467]
 [ 1.         -0.14023015  1.15571977]
 [ 1.         -0.8143543  -1.22087578]]
         area
0   -0.201824
1   -0.201824
2   -0.201824
3   -0.201824
4   -0.201824
..        ...
512 -0.100655
513  0.651044
514 -0.026506
515 -0.201824
516 -0.201824

[517 rows x 1 columns]


# Modelo 

Nota: Antes añadir Añadir una columna de nx1 a X con valor 1.

$h(X) = X*W^t$


In [15]:
def h(X, W):
    return np.dot(X,np.transpose(W))

# Loss function
Nota: La función de pérdida no cambia, solo la llamada a la función h
$\mathcal{L} =  ||Y - XW^t||_2^2$


Norma L2

In [16]:
def Error(X, W, Y, rho, lam):
    ridge_component = (1- rho)*lam*np.sum(np.power(W, 2))
    lasso_component = rho*(lam*np.sum(np.abs(W)))
    n = X.shape[0]
    normal_L2 = np.sum(np.power(Y - h(X, W), 2)) / (2 * n)
    loss = normal_L2 + ridge_component + lasso_component
    return normal_L2

# Cálculo de derivadas
Nota: Intente resolver este algoritmo desde un punto de vista matricial.

$dw_j = \frac{1}{m}\sum_{i=0}^m(y_i - h(x_i))(-x_{i,j})$

In [17]:
def derivada(X, W, Y, lam, rho):
    n = X.shape[0]
    normal_L2_derivate = np.dot((Y - h(X,W)).transpose(), (-1 * X)) / n
    ridge_derivate = 2*lam*(1-rho)*W
    lasso_derivate = np.sign(W)*rho*lam
    dW = normal_L2_derivate  + ridge_derivate + lasso_derivate
    return normal_L2_derivate

# Actualiación de parámetros 

Recuerde: $\frac{\partial L}{\partial w}$ representa un vector con todas las derivadas de la función de pérdida con rescto a W. 

$W  = W - \alpha*\frac{\partial L}{\partial W} $


In [18]:
def update(W,  dW, alpha): 
    return W - alpha * dW

# Training

In [19]:
def train(X, Y, umbral, alfa):
    # Creamos un vector fila W aleatoria con el tamaño de columnas de X
    X_columns = X.shape[1]
    W = np.random.rand(1,X_columns)
    rho = 0
    lam = 10
    L = Error(X,W,Y,rho, lam)
    print(L[0])
    loss = []
    while (L[0] > umbral):
        dW = derivada(X, W, Y, lam, rho)
        W = update(W, dW, alfa)
        L = Error(X, W,Y, rho, lam)
        print(W)
        print("--------------")
        loss.append(L)
    return W, loss


def Plot_Loss(epochs,loss):
   plt.plot(epochs, loss)

## Plot

In [20]:
# import numpy as np
# import matplotlib.pyplot as plt

# plt.rcParams["figure.figsize"] = [7.00, 3.50]
# plt.rcParams["figure.autolayout"] = True

# x = np.linspace(-10, 10, 100)
# y = np.linspace(-10, 10, 100)

# x, y = np.meshgrid(x, y)
 
# eq = 0.1 * x + 40 * y + 100.09  # ecuacion del plano: x_1w_1 + x_2w_2 + b

# fig = plt.figure()

# ax = fig.gca(projection='3d')

# ax.plot_surface(x, y, eq)

# plt.show()

# Testing

In [21]:
def plot3D(w_0, w_1, w_2, X, Y):
    plt.rcParams["figure.figsize"] = [7.00, 3.50]
    plt.rcParams["figure.autolayout"] = True

    x = np.linspace(-10, 10, 100)
    y = np.linspace(-10, 10, 100)

    x, y = np.meshgrid(x, y)
    
    eq = 0.1 * w_0 + 40 * w_1 + w_2 # ecuacion del plano: x_1w_1 + x_2w_2 + b

    fig = plt.figure()

    ax = fig.gca(projection='3d')

    ax.plot_surface(x, y, eq)

    plt.show()

In [22]:
umbral = 0.25
alpha = 2

W = train(X, Y, umbral, alpha)
#plot3D(W[0],W[1],W[2],X,Y)

0.6529886373160981
[[-0.11305727  0.11043861  0.00026048]]
--------------
[[ 0.11305727 -0.26048215  0.31131343]]
--------------
[[-0.11305727  0.43646175 -0.38901952]]
--------------
[[ 0.11305727 -0.99505489  1.04230527]]
--------------
[[-0.11305727  1.93773751 -1.890498  ]]
--------------
[[ 0.11305727 -4.07119019  4.11842909]]
--------------
[[-0.11305727  8.24033294 -8.19309407]]
--------------
[[  0.11305727 -16.98440224  17.0316411 ]]
--------------
[[ -0.11305727  34.69785165 -34.65061278]]
--------------
[[  0.11305727 -71.19247178  71.23971065]]
--------------
[[-1.13057273e-01  1.45763248e+02 -1.45716009e+02]]
--------------
[[ 1.13057273e-01 -2.98751254e+02  2.98798493e+02]]
--------------
[[-1.13057273e-01  6.12002068e+02 -6.11954829e+02]]
--------------
[[ 1.13057273e-01 -1.25401492e+03  1.25406216e+03]]
--------------
[[-1.13057273e-01  2.56921506e+03 -2.56916782e+03]]
--------------
[[ 1.13057273e-01 -5.26409384e+03  5.26414108e+03]]
--------------
[[-1.13057273e-01  1

/opt/python/envs/default/lib/python3.8/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
<ipython-input-16-d4a31dd5c6b9>:2: RuntimeWarning: overflow encountered in double_scalars
  ridge_component = (1- rho)*lam*np.sum(np.power(W, 2))
<ipython-input-16-d4a31dd5c6b9>:2: RuntimeWarning: overflow encountered in power
  ridge_component = (1- rho)*lam*np.sum(np.power(W, 2))
<ipython-input-16-d4a31dd5c6b9>:3: RuntimeWarning: invalid value encountered in double_scalars
  lasso_component = rho*(lam*np.sum(np.abs(W)))
